This notebook includes the calculation of the three-level segmented grid charges and the optimization of empirical household energy consumption with flat and dynamic energy prices for this case.

### Requirements

Load the necessary packages to complete the optimization.

In [1]:
import pandas as pd
import numpy as np
from gurobipy import *

### Methods

The first method is used in the optimization to determine the start and end points of each charging session, as well as the overall charged energy within one session. The second method is used to write the results of the optimization into a dataframe, enabling further analysis.

In [2]:
#Function to calculate start point, end point, and total energy charged for each charging session
def chargPerUser(load,percent):
    
    #List to store information about charging sessions
    charging_sessions = []
    
    total_energy_charged = 0
    b=False
    b1=False

    #Iterate through all time steps
    for i in range(len(percent)):
        
        #If the plugin hour is not a full hour
        if percent[i] < 1 and percent[i] != 0 and b == False:
            total_energy_charged += load[i] #Add charged energy
            start_index = i #Save the start point
            b = True
            
        #If the plugin hour is a full hour   
        elif percent[i]==1 and percent[i-1]==0:
            total_energy_charged += load[i]
            start_index = i
            b = True
        
        #Full charging hour between plugin and plugout
        elif percent[i] == 1:
            total_energy_charged += load[i]
            b = True
            b1 = True
            
        #Plugout after hours of full charging and the end hour is not a full hour   
        elif percent[i] < 1 and percent[i] != 0 and b == True and b1 == True and percent[i+1] == 0:
            total_energy_charged += load[i]
            end_index = i #Save the end point
            charging_sessions.append({'start_index': start_index, 'end_index': end_index, 'total_energy': total_energy_charged})
            total_energy_charged = 0
            b = False
            b1 = False
            
        #Plugout after no hour of full charging and the end hour is not a full hour 
        elif percent[i] < 1 and percent[i] != 0 and b == True and b1 == False and percent[i+1] == 0:
            total_energy_charged += load[i]
            end_index = i
            charging_sessions.append({'start_index': start_index, 'end_index': end_index, 'total_energy': total_energy_charged})
            total_energy_charged = 0
            b = False
            
        #Plugout in the same hour as plugin
        elif percent[i] == 0 and b == True and b1 == False:
            end_index = start_index
            charging_sessions.append({'start_index': start_index, 'end_index': end_index, 'total_energy': total_energy_charged})
            total_energy_charged = 0
            b = False
        
        #Plugout after hours of full charging and the end hour is a full hour 
        elif percent[i] == 0 and b == True and b1 == True:
            end_index = i-1
            charging_sessions.append({'start_index': start_index, 'end_index': end_index, 'total_energy': total_energy_charged})
            total_energy_charged = 0 
            b = False
            b1 = False
        
        #If no energy is charged or if several not full hours follow each other
        else:
            total_energy_charged += load[i]
            
    return(charging_sessions)

In [3]:
#Write resluts of optimization in dataframe
def get_results_in_df(variableNames, n_timesteps, model): #  "def name(argumente)" zeigt den Anfang einer Funktion an, die ich später beliebig aufrufen kann
   

    results_df = pd.DataFrame(columns = variableNames, index = [t for t in range(n_timesteps)] )   # Hier wird ein leeres DataFrame erstellt, dass als Spalteneinträge die Namen der Gutobi-variablen hat und als Zeilen die Zeitschritte der Entscheidungsvariablen

    for n in variableNames:                                         #Iteration über alle Zielvariablen
        for t in range(n_timesteps):                                #Iteration über alle Zeitschritte
            VarName = n + f"[{t}]"                                  #Hier wird ein String erstellt, der die Form n[t] hat. Mit diesem wird später die Zielvariable ausgelesen
            try:                                                    #try - except ist eine hilfreiche Methdoe, um Fehler abzufangen. Wenn in der nächsten Zeile ein error passiert, bspw. wegen eines Schreibfehlers in meinen zielvariablen, wird die Funktion nicht abgebrochen sondern weiter ausgeführt.
                results_df.loc[t][n] =  model.getVarByName(VarName).x    #Auslesen der Zielvaribale
            except:
                pass
            
    return results_df

### Dictionaries

The following dictionaries are used to reference column names by indexing:

In [4]:
#Dictionary for households
hh_dic={1:"SFH3_HH",2:"SFH4_HH",3:"SFH5_HH",4:"SFH7_HH",5:"SFH9_HH",6:"SFH12_HH",7:"SFH14_HH",8:"SFH16_HH",9:"SFH18_HH",10:"SFH19_HH",
        11:"SFH20_HH",12:"SFH21_HH",13:"SFH22_HH",14:"SFH27_HH",15:"SFH28_HH",16:"SFH29_HH",17:"SFH30_HH",18:"SFH32_HH",19:"SFH34_HH",
        20:"SFH36_HH",21:"SFH38_HH",22:"SFH39_HH"}

In [5]:
#Dictionary for heatpumps
hp_dic={1:"SFH3_HP",2:"SFH4_HP",3:"SFH5_HP",4:"SFH7_HP",5:"SFH9_HP",6:"SFH12_HP",7:"SFH14_HP",8:"SFH16_HP",9:"SFH18_HP",10:"SFH19_HP",
        11:"SFH20_HP",12:"SFH21_HP",13:"SFH22_HP",14:"SFH27_HP",15:"SFH28_HP",16:"SFH29_HP",17:"SFH30_HP",18:"SFH32_HP",19:"SFH34_HP",
        20:"SFH36_HP",21:"SFH38_HP",22:"SFH39_HP"}

### Optimization

The following method describes the optimization of empirical household consumption with three-level segmented grid charges to minimize total costs. HH consumption remains inflexible. Shifting HP consumption is restricted by a maximum number of blocked time steps (sigma_max) in each interval of length 'length'. The total consumption over an interval must match the empirical consumption within that interval. EV shifting is freely possible between plug-in and plug-out times, as long as the maximum charging power (ev_max) of the wallbox is not surpassed and the charged energy for the session matches the empirical data.

In [6]:
#Optimization of one household with hp flexibility within intervals and flexible ev between plugin and plugout with flat energy prices
def optimized_HP_EV_flat(hh, hp, ev_user, peak_ev, flexibility, length, p_flat, T1,T2,lambda1,lambda2,lambda3):
    
    l = len(hh)
    l_ev= len(ev_user.columns)//2
    ev_max = peak_ev
    hp_max = max(hp)
    sigma_max = flexibility
    part = length
    epsilon = 0.00000000001
    
    #1. create Model
    model  = Model("simpleHP")
        

    #2. define Variables
    
    #2.1 hp load over all time steps
    d_hp = model.addVars(l, vtype=GRB.CONTINUOUS, lb=0, ub=hp_max, name="d_hp")
    
    #2.2 procentual hp shifts over all time steps
    sigma = model.addVars(l,vtype=GRB.BINARY, name="sigma")
    
    #2.3 ev charging load over all time steps
    d_ev = model.addVars(l, l_ev, vtype=GRB.CONTINUOUS, lb=0, ub=ev_max, name="d_ev")
    
    #2.4 total consumption over all time steps
    d = model.addVars(l, vtype=GRB.CONTINUOUS, lb=0, name="d")

    #2.5
    p_grid = model.addVars(l, vtype=GRB.CONTINUOUS, lb=0, name="p_grid")

    condition_1 = model.addVars(l, vtype=GRB.BINARY, name="condition_1")
    condition_2 = model.addVars(l, vtype=GRB.BINARY, name="condition_2")
    condition_3 = model.addVars(l, vtype=GRB.BINARY, name="condition_3")

    #3. constraints
    
    #3.1 total consumption over all time steps as sum of hp, hh and all evs
    model.addConstrs(d[t] == d_hp[t]+hh[t]+quicksum(d_ev[t,u] for u in range(l_ev)) for t in range(l))
    
    #HP
    #3.2 calculation of hp load
    model.addConstrs(d_hp[t] >= (1-sigma[t])*hp[t] for t in range(l))
    
    #3.3 hp flexibility is restricted to flex_hours through binary variable sigma
    model.addConstrs(quicksum(sigma[k] for k in range(part*i,part*i+part)) <= sigma_max for i in range(l//part))

    #3.4 hp load over time part must at least be empirical hp
    model.addConstrs(quicksum(d_hp[k] for k in range(part*i,part*i+part)) >= quicksum(hp[k] for k in range(part*i,part*i+part)) for i in range(l//part))
    
    
    #EV
    #iterate through all evs (users) that charge at this garage
    for u in range(l_ev):
        ev = ev_user.iloc[:,u*2]
        ev_perc = ev_user.iloc[:,u*2+1]
        
        #3.5 in every timestep ev power demand must be smaller than ev_max
        model.addConstrs(d_ev[t,u] <= ev_perc[t]*ev_max for t in range(l))
        
        #get for every charging session start and end time and the total charged energy
        charging_sessions=chargPerUser(ev,ev_perc)
        list_length = len(charging_sessions)
        
        #iterate through all charging sessions of one user
        for i in range (list_length):
            start_index = charging_sessions[i]['start_index']
            end_index = charging_sessions[i]['end_index']
            total_energy = charging_sessions[i]['total_energy']
            
            #3.6 the charged power between start and end must be equal to the total charged energy
            model.addConstr(quicksum(d_ev[t,u] for t in range(start_index,end_index+1)) == total_energy)
   
    #3.7 Add constraints to enforce conditions based on d[t]
    for t in range(l):
        model.addGenConstrIndicator(condition_1[t], True, T1-d[t] >= 0.0)
        model.addGenConstrIndicator(condition_1[t], False, T1-d[t] <= 0.0+epsilon)
        model.addGenConstrIndicator(condition_2[t], True, condition_1[t]+condition_3[t] == 0.0)
        model.addGenConstrIndicator(condition_2[t], False, condition_1[t]+condition_3[t] == 1.0)
        model.addGenConstrIndicator(condition_3[t], True, T2-d[t] <= 0.0+epsilon)
        model.addGenConstrIndicator(condition_3[t], False, T2-d[t] >= 0.0)
        
            
    #4. optimize
    objective_cost = quicksum((d[t]*p_flat 
                               + condition_1[t] * (d[t] * lambda1) 
                               + condition_2[t]* (T1 * lambda1 + (d[t] - T1) * lambda2) 
                               + condition_3[t] * (T1 * lambda1 + (T2 - T1) * lambda2 + (d[t] - T2) * lambda3)) 
                              for t in range(l))

    #4.1 set the costs as the variable to be minimized
    model.setObjective(objective_cost,GRB.MINIMIZE)

    #4.2 perform optimization
    model.optimize()
    
    #5 return results
    #5.1 get objective value
    cost = model.getObjective().getValue()
    
    #get values for total load d for every hour
    var =["d","condition_1","condition_2","condition_3","d_hp"]
    load_after_shift = get_results_in_df(var,l, model)
    
    return cost, load_after_shift

In [7]:
#Optimization of one household with hp flexibility within intervals and flexible ev between plugin and plugout with dynamic energy prices
def optimized_HP_EV(hh, hp, ev_user, peak_ev, flexibility, part, p_flex, T1,T2,lambda1,lambda2,lambda3):
    
    l = len(hh)
    l_ev= len(ev_user.columns)//2
    ev_max = peak_ev
    hp_max = max(hp)
    sigma_max = flexibility
    part = part
    epsilon = 0.00000000001
    
    #1. create Model
    model  = Model("simpleHP")
        

    #2. define Variables
    
    #2.1 hp load over all time steps
    d_hp = model.addVars(l, vtype=GRB.CONTINUOUS, lb=0, ub=hp_max, name="d_hp")
    
    #2.2 procentual hp shifts over all time steps
    sigma = model.addVars(l,vtype=GRB.BINARY, name="sigma")
    
    #2.3 ev charging load over all time steps
    d_ev = model.addVars(l, l_ev, vtype=GRB.CONTINUOUS, lb=0, ub=ev_max, name="d_ev")
    
    #2.4 total consumption over all time steps
    d = model.addVars(l, vtype=GRB.CONTINUOUS, lb=0, name="d")

    #2.5
    p_grid = model.addVars(l, vtype=GRB.CONTINUOUS, lb=0, name="p_grid")

    condition_1 = model.addVars(l, vtype=GRB.BINARY, name="condition_1")
    condition_2 = model.addVars(l, vtype=GRB.BINARY, name="condition_2")
    condition_3 = model.addVars(l, vtype=GRB.BINARY, name="condition_3")

    #3. constraints
    
    #3.1 total consumption over all time steps as sum of hp, hh and all evs
    model.addConstrs(d[t] == d_hp[t]+hh[t]+quicksum(d_ev[t,u] for u in range(l_ev)) for t in range(l))
    
    #HP
    #3.2 calculation of hp load
    model.addConstrs(d_hp[t] >= (1-sigma[t])*hp[t] for t in range(l))
    
    #3.3 hp flexibility is restricted to flex_hours through binary variable sigma
    model.addConstrs(quicksum(sigma[k] for k in range(part*i,part*i+part)) <= sigma_max for i in range(l//part))

    #3.4 hp load over time part must at least be empirical hp
    model.addConstrs(quicksum(d_hp[k] for k in range(part*i,part*i+part)) >= quicksum(hp[k] for k in range(part*i,part*i+part)) for i in range(l//part))
    
    
    #EV
    #iterate through all evs (users) that charge at this garage
    for u in range(l_ev):
        ev = ev_user.iloc[:,u*2]
        ev_perc = ev_user.iloc[:,u*2+1]
        
        #3.5 in every timestep ev power demand must be smaller than ev_max
        model.addConstrs(d_ev[t,u] <= ev_perc[t]*ev_max for t in range(l))
        
        #get for every charging session start and end time and the total charged energy
        charging_sessions=chargPerUser(ev,ev_perc)
        list_length = len(charging_sessions)
        
        #iterate through all charging sessions of one user
        for i in range (list_length):
            start_index = charging_sessions[i]['start_index']
            end_index = charging_sessions[i]['end_index']
            total_energy = charging_sessions[i]['total_energy']
            
            #3.6 the charged power between start and end must be equal to the total charged energy
            model.addConstr(quicksum(d_ev[t,u] for t in range(start_index,end_index+1)) == total_energy)
   
    #3.7 Add constraints to enforce conditions based on d[t]
    for t in range(l):
        model.addGenConstrIndicator(condition_1[t], True, T1-d[t] >= 0.0)
        model.addGenConstrIndicator(condition_1[t], False, T1-d[t] <= 0.0+epsilon)
        model.addGenConstrIndicator(condition_2[t], True, condition_1[t]+condition_3[t] == 0.0)
        model.addGenConstrIndicator(condition_2[t], False, condition_1[t]+condition_3[t] == 1.0)
        model.addGenConstrIndicator(condition_3[t], True, T2-d[t] <= 0.0+epsilon)
        model.addGenConstrIndicator(condition_3[t], False, T2-d[t] >= 0.0)
        
            
    #4. optimize
    objective_cost = quicksum((d[t]*p_flex.iloc[t,0] 
                                + condition_1[t] * (d[t] * lambda1) 
                                + condition_2[t]* (T1 * lambda1 + (d[t] - T1) * lambda2) 
                                + condition_3[t] * (T1 * lambda1 + (T2 - T1) * lambda2 + (d[t] - T2) * lambda3)) 
                                for t in range(l))


    #4.1 set the costs as the variable to be minimized
    model.setObjective(objective_cost,GRB.MINIMIZE)

    #4.2 perform optimization
    model.optimize()
    
    #5 return results
    #5.1 get objective value
    cost = model.getObjective().getValue()
    
    #get values for total load d for every hour
    var =["d","condition_1","condition_2","condition_3","d_hp"]
    load_after_shift = get_results_in_df(var,l, model)
    
    return cost, load_after_shift

### Read in data

Read in EV charging data, household and heat pump consumption patterns, maximum charging power of each wallbox, day-ahead market prices, garage IDs, the total load in the empirical case as well as the total costs in the empirical scenario. All data has been pre-processed beforehand.

In [8]:
ev_hourly = pd.read_csv('Hourly_EV_Charging.csv')
hh_hp = pd.read_csv('Hourly_HH_HP.csv')
peak_ev = pd.read_csv('Min_Peak_Load.csv')
p_flex = pd.read_csv('Hourly_Flexible_Prices.csv')
unique_garage = pd.read_csv('Unique_Garage.csv')
load_before_shift = pd.read_csv('load_before_shift.csv',index_col=0)
cost_before_shift_flat = pd.read_csv('cost_before_shift.csv', index_col=0)

Calculate Thresholds for Three-Level-Segmented Tariff

In [12]:
#Determine min and max consumption of all households over the time period and then calculate the average
consumption_total = load_before_shift.sum()
min_con=min(consumption_total)
max_con=max(consumption_total)
average = (min_con+max_con)/2
hourly_av=average/(9504)
beta1 = 2
beta2 = 12

In [13]:
T1 = math.ceil(hourly_av * 10) / 10
T2 = math.ceil(hourly_av* 3 * 10) / 10

Calculate costs for lambda1, lambda2, and lambda3 using empirical data so that they match the DSO revenue in the baseline case.

In [9]:
cost_before_shift_flat_list=cost_before_shift_flat['cost_before_shift'].tolist()

After approximating the values for lambda1, lambda2, and lambda3, these results are used as prices in the optimization. The difference (diffsum) at the end of the while loop was equal to 8.00270510651012.

In [10]:
lambda1 = 0.17463244917878995
lambda2 =0.3492648983575799
lambda3 =2.0955893901454794

Calculate the energy costs per kWh in case of flat energy tariff in combination with three-level segmented grid charges

In [14]:
cost_before_shift_3L_flex = []
for i in range(len(peak_ev)):
    cost = 0
    for k in range(len(p_flex)):
        p_grid=0
        if load_before_shift.iloc[k,i]<=T1:
            p_grid = load_before_shift.iloc[k,i] * lambda1 
        elif load_before_shift.iloc[k,i]<=T2 and load_before_shift.iloc[k,i]>T1:
            p_grid = T1 * lambda1 + (load_before_shift.iloc[k,i] - T1) * lambda2
        elif load_before_shift.iloc[k,i]>T2:
            p_grid = T1 * lambda1 + (T2 - T1) * lambda2 + (load_before_shift.iloc[k,i] - T2) * lambda3
        cost += p_grid
    cost_before_shift_3L_flex.append(cost)

In [16]:
p_flat = (sum(cost_before_shift_flat_list)-sum(cost_before_shift_3L_flex))/load_before_shift.sum().sum()

### Perform optimization

Perform optimization for each HH, first for flat energy prices and then for dynamic energy prices, to determine the new household consumption patterns for each scenario.

In [19]:
load_after_shift_flat = pd.DataFrame()
cost_after_shift_flat = []

#Iterate over all households
for i in range(len(peak_ev)):
    
    #Select all users per garage
    selected_columns = ev_hourly.filter(regex=f'^{unique_garage.iloc[i,0]}-', axis=1)
    
    #Perform optimization
    cost, load = optimized_HP_EV_flat(hh_hp[hh_dic[i+1]],hh_hp[hp_dic[i+1]], selected_columns, peak_ev.iloc[i,0],2,4,p_flat,T1,T2,lambda1,lambda2,lambda3)
    
    #Save total costs
    cost_after_shift_flat.append(cost)
    
    #Store shifted loads per household
    load_after_shift_flat[f"{hh_dic[i+1]}_d"] = load.iloc[:,0]

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 10.0 (19045.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 52565 rows, 95040 columns and 118818 nonzeros
Model fingerprint: 0x64235d6d
Model has 28512 quadratic objective terms
Model has 57024 general constraints
Variable types: 57024 continuous, 38016 integer (38016 binary)
Coefficient statistics:
  Matrix range     [1e-02, 6e+00]
  Objective range  [4e-02, 9e+00]
  QObjective range [3e-01, 4e+00]
  Bounds range     [1e+00, 7e+00]
  RHS range        [1e-02, 4e+01]
  GenCon rhs range [1e+00, 5e+00]
  GenCon coe range [1e+00, 1e+00]
Presolve added 13172 rows and 0 columns
Presolve removed 0 rows and 39009 columns
Presolve time: 2.49s
Presolved: 94085 rows, 84379 columns, 244373 nonzeros
Variable types: 46527 continuous, 37852 integer (37852 binary)
Found heuristic solut

In [20]:
load_after_shift = pd.DataFrame()
cost_after_shift = []
#hp = pd.DataFrame() #if HP load is needed

#Iterate over all households
for i in range(len(peak_ev)):
    
    #Select all users per garage
    selected_columns = ev_hourly.filter(regex=f'^{unique_garage.iloc[i,0]}-', axis=1)
    
    #Perform optimization
    cost, load = optimized_HP_EV(hh_hp[hh_dic[i+1]],hh_hp[hp_dic[i+1]], selected_columns, peak_ev.iloc[i,0],2,4,p_flex,T1,T2,lambda1,lambda2,lambda3)
    
    #Save total costs
    cost_after_shift.append(cost)
    
    #Store shifted loads per household
    load_after_shift[f"{hh_dic[i+1]}_d"] = load.iloc[:,0]
    #hp[f"{hh_dic[i+1]}_d"] = load.iloc[:,4]

Gurobi Optimizer version 11.0.0 build v11.0.0rc2 (win64 - Windows 10.0 (19045.2))

CPU model: 11th Gen Intel(R) Core(TM) i7-1165G7 @ 2.80GHz, instruction set [SSE2|AVX|AVX2|AVX512]
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 52565 rows, 95040 columns and 118818 nonzeros
Model fingerprint: 0x2ab4321a
Model has 28512 quadratic objective terms
Model has 57024 general constraints
Variable types: 57024 continuous, 38016 integer (38016 binary)
Coefficient statistics:
  Matrix range     [1e-02, 6e+00]
  Objective range  [1e-05, 9e+00]
  QObjective range [3e-01, 4e+00]
  Bounds range     [1e+00, 7e+00]
  RHS range        [1e-02, 4e+01]
  GenCon rhs range [1e+00, 5e+00]
  GenCon coe range [1e+00, 1e+00]
Presolve added 13173 rows and 0 columns
Presolve removed 0 rows and 39001 columns
Presolve time: 2.40s
Presolved: 94086 rows, 84387 columns, 244382 nonzeros
Variable types: 46535 continuous, 37852 integer (37852 binary)
Found heuristic solut

### Save Data

In [21]:
#save as a dataframe
cost_after_shift_flat_df = pd.DataFrame(cost_after_shift_flat, columns=['cost_after_shift'])
cost_after_shift_df = pd.DataFrame(cost_after_shift, columns=['cost_after_shift'])

cost_after_shift_flat_df.to_csv('cost_after_shift_3Level_flat.csv')
load_after_shift_flat.to_csv('load_after_shift_3Level_flat.csv')
#hp.to_csv('hp_after_shift_3Level.csv')
cost_after_shift_df.to_csv('cost_after_shift_3Level_dyn.csv')
load_after_shift.to_csv('load_after_shift_3Level_dyn.csv')